In [1]:

import pandas as pd
import traitlets
import ipyvuetify as v
import json

class PandasDataFrame(v.VuetifyTemplate):
    """
    Vuetify DataTable rendering of a pandas DataFrame
    
    Args:
        data (DataFrame) - the data to render
        title (str) - optional title
    """
    
    headers = traitlets.List([]).tag(sync=True, allow_null=True)
    data_headers = traitlets.List([]).tag(sync=True, allow_null=True)
    items = traitlets.List([]).tag(sync=True, allow_null=True)
    search = traitlets.Unicode('').tag(sync=True)
    col = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('DataFrame').tag(sync=True)
    index_col = traitlets.Unicode('').tag(sync=True)
    dialog = traitlets.Bool().tag(sync=True)
    formTitle = traitlets.Unicode('New Item').tag(sync=True)
    editedItem_species = traitlets.Unicode('').tag(sync=True)
    editedItems = traitlets.List([]).tag(sync=True, allow_null=True)
    
    dialog_fields = traitlets.Dict(traitlets.Unicode(''))
    template = traitlets.Unicode('''
        <template>
          <v-card>
            <v-card-title>
              <span class="title font-weight-bold">{{ title }}</span>
              <v-spacer></v-spacer>

            </v-card-title>
            <v-data-table
                :headers="headers"
                :items="items"
                :search="search"
                :item-key="index_col"
                :footer-props="{'items-per-page-options': [25, 50, 250, 500]}"
            >
            
                <template v-slot:top>
                  <v-toolbar flat color="white">
                    <v-toolbar-title>My CRUD</v-toolbar-title>
                    <v-divider
                      class="mx-4"
                      inset
                      vertical
                    ></v-divider>
                    <v-spacer></v-spacer>
                    <v-text-field
                        v-model="search"
                        append-icon="mdi-magnify"
                        label="Search ..."
                        single-line
                        hide-details
                     ></v-text-field>
                    <v-spacer></v-spacer>
                
                    <v-dialog v-model="dialog" max-width="500px">
                      <template v-slot:activator="{ on, attrs }">
                        <v-btn
                          color="primary"
                          dark
                          class="mb-2"
                          v-bind="attrs"
                          v-on="on"
                        >New Item</v-btn>
                      </template>
                      
                      <v-card>
                        <v-card-title>
                          <span class="headline">{{ formTitle }}</span>
                        </v-card-title>

                        <v-card-text>
                          <v-container>
                            <v-row>
                              
                                <v-col v-for="col in editedItems" cols="12" sm="6" md="4">
                                  <v-text-field v-model="col.value" v-bind:label=col.name></v-text-field>
                                </v-col>
                              
                             </v-row>
                          </v-container>
                        </v-card-text>

                        <v-card-actions>
                          <v-spacer></v-spacer>
                          <v-btn color="blue darken-1" text @click="close">Cancel</v-btn>
                          <v-btn color="blue darken-1" text @click="save">Save</v-btn>
                        </v-card-actions>
                      </v-card>
                      
                      
                    </v-dialog>
                  </v-toolbar>
                </template>
   
    
                <template v-slot:item.actions="{ item }">
                  <v-icon
                    small
                    class="mr-2"
                    @click="editItem(item)"
                  >
                    mdi-pencil
                  </v-icon>
                  <v-icon
                    small
                    @click="deleteItem(item)"
                  >
                    mdi-delete
                  </v-icon>
                </template>
                <template v-slot:no-data>
                  <v-alert :value="true" color="error" icon="mdi-alert">
                    Sorry, nothing to display here :(
                  </v-alert>
                </template>
                <template v-slot:no-results>
                    <v-alert :value="true" color="error" icon="mdi-alert">
                      Your search for "{{ search }}" found no results.
                    </v-alert>
                </template>
                <template v-slot:items="rows">
                  <td v-for="(element, label, index) in rows.item"
                      @click=cell_click(element)
                      >
                    {{ element }}
                  </td>
                </template>
            </v-data-table>
          </v-card>
        </template>
        ''').tag(sync=True)
    
    def __init__(self, *args, 
                 data=pd.DataFrame(), 
                 title=None,
                 **kwargs):
        super().__init__(*args, **kwargs)
        data = data.reset_index()
        self.index_col = data.columns[0]
        self.data_headers = [{
              "text": col,
              "value": col
            } for col in data.columns]
        self.data_headers[0].update({'align': 'left', 'sortable': True})
        self.headers = self.data_headers + [{"text": "Actions", "value": "actions"}]
        self.items = json.loads(
            data.to_json(orient='records'))
        if title is not None:
            self.title = title
        self.edited_item = False
#         self.dialog_fields['species'] = ''
        self.editedItems = [{'name':header['text'], 'value':''} for header in self.data_headers[1:]]
            
            
    def vue_deleteItem(self, data):
        idx = self.items.index(data)
        self.items = self.items[:idx] + self.items[idx+1:]

    def vue_editItem(self, data):
#         self.title = str(data)
        self.editedItems = [{'name':header['text'], 'value':data[header['text']]} 
                            for header in self.data_headers[1:]] #[{'name':'species', 'value':data['species']}]
        self.dialog = True
        self.edited_item = data
        self.formTitle = "Edit Item"
#         self.editedItem_species = data['species']
#         self.editedItems[0]['species'] = data['species']
        
    
    def vue_close(self, data):
        self.dialog = False
        self.editedItems = [{'name':'species', 'value':''}]
        self.edited_item = False # restore to default value
        self.formTitle = "New Item" # restore to default value
        
    def vue_save(self, data):
        
        if self.edited_item:
            #it's an edit
            
            idx = self.items.index(self.edited_item)
            edited_item = {'index': self.items[idx]['index'],}
            edited_item = {'index': self.items[idx]['index']}
            edited_item.update({item['name']:item['value'] for item in self.editedItems})
                                                                     
                                                                     
#                         {'sepal_length': self.items[idx]['sepal_length'],
#                         'sepal_width': self.items[idx]['sepal_width'],
#                         'petal_length': self.items[idx]['petal_length'],
#                         'petal_width': self.items[idx]['petal_width'],
# #                         'species': self.editedItem_species}
#                         'species': self.editedItems[0]['value']}
#                                                                     )
            self.items = self.items[:idx] + [edited_item] + self.items[idx+1:]
            
        else:
            # it's a new item
            
            new_item = {'index': self.items[-1]['index']+1}
            new_item.update({item['name']:item['value'] for item in self.editedItems})
            self.items = self.items + [new_item]
        self.dialog = False
        self.edited_item = False # restore to default value
        self.formTitle = "New Item" # restore to default value
        self.editedItems = [{'name':header['text'], 'value':''} for header in self.data_headers[1:]] # restore to default value
    
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv').iloc[:5,:]
test = PandasDataFrame(data = iris, title='Iris')
test

PandasDataFrame(data_headers=[{'text': 'index', 'value': 'index', 'align': 'left', 'sortable': True}, {'text':…